In [1]:
import pandas as pd
from sklearn.metrics import f1_score, fbeta_score
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task

import dill
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, recall_score, precision_score, fbeta_score, confusion_matrix
from tqdm import tqdm


In [2]:
df_train = pd.read_csv('df_train_ADASYN_MinMax.csv')
df_test = pd.read_csv('df_test_ADASYN_MinMax.csv')
print(df_train.shape)
print(df_test.shape)
df_train.head()

(258448, 314)
(46412, 316)


,hdb_bki_total_oth_cnt,stratsegfactor_Инвестиционные продукты,amount_by_category_90d__summarur_amt__SUM__cashflowcategory_name__dom_kvartira,clientoutflowstatus_OUTFLOW,amount_by_category_90d__summarur_amt__SUM__cashflowcategory_name__kino,amount_by_category_90d__summarur_amt__SUM__cashflowcategory_name__kofejni,profit_income_out_rur_amt_12m,amount_by_category_30d__summarur_amt__SUM__cashflowcategory_name__meditsinskie_tsentry,stratsegfactor_Остатки на счетах,transaction_category_transport_percent_cnt_2m,...,transaction_category_fastfood_sum_cnt_m2,transaction_category_sporting_goods_percent_cnt_2m,transaction_category_auto_services_percent_cnt_2m,amount_by_category_30d__summarur_amt__SUM__cashflowcategory_name__hobbi,hdb_relend_client_credits_count_grade_nan,hdb_bki_total_pil_last_days,amount_by_category_90d__summarur_amt__SUM__cashflowcategory_name__vydacha_nalichnyh_v_bankomate,depos_rur_amt_3m_avg,transaction_category_other_retail_purchase_inc_amt_2m,target
0,0.0,0.0,0.0,0.0,0.0,0.000546,0.088663,0.0,1.0,0.041667,...,0.002817,0.0,0.0,0.0,0.0,0.340916,0.000664,0.000000,0.324172,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.088548,0.0,1.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.823967,0.000000,0.000000,0.000000,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.089190,0.0,0.0,0.027027,...,0.002817,0.0,0.0,0.0,0.0,1.000000,0.000000,0.002541,0.139190,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.089326,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.006393,0.000000,0.000000,0.000000,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.088660,0.0,1.0,0.125000,...,0.008451,0.0,0.0,0.0,0.0,0.480672,0.000000,0.000000,0.000000,0.0


In [3]:
df_test=df_test.drop(columns=['client_id','dt'])

In [4]:
df_test['target'].mean()

0.06468154787554943

In [9]:
automl = TabularAutoML(task = Task('binary', metric = lambda y_true, y_pred: fbeta_score(y_true, (y_pred > 0.5)*1, beta=2)), cpu_limit=28, general_params = {'nested_cv': False},reader_params = {'cv': 3, 'random_state': 42},)
oof_pred = automl.fit_predict(df_train, roles = {'target': 'target', 'drop': ['client_id', 'dt']}, verbose=100)


[05:19:10] Stdout logging level is DEBUG.
[05:19:10] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[05:19:10] Task: binary

[05:19:10] Start automl preset with listed constraints:
[05:19:10] - time: 3600.00 seconds
[05:19:10] - CPU: 28 cores
[05:19:10] - memory: 16 GB

[05:19:10] Train data shape: (258448, 314)

[05:19:57] Feats was rejected during automatic roles guess: []
[05:19:59] Layer 1 train process start. Time left 3551.16 secs
[05:21:06] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[05:21:06] Training params: {'tol': 1e-06, 'max_iter': 100, 'cs': [1e-05, 5e-05, 0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100, 500, 1000, 5000, 10000, 50000, 100000], 'early_stopping': 2, 'categorical_idx': [449, 450, 451], 'embed_sizes': array([10,  9,  9], dtype=int32), 'data_size': 452}
[05:21:08] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[05:21:16] Linear model: C = 1e-05 score = 0.93351641

Подбираем порог по F2 на трейне

In [17]:
f2_score_max = 0
tresh = 0
for tresh in list(np.arange(0.05,0.95, 0.05)):
    pred_proba = automl.predict(df_train).data[:, 0]
    pred = (pred_proba > 0.5)*1
    if fbeta_score(df_train['target'], pred, beta=2) > f2_score_max:
        f2_score_max = fbeta_score(df_train['target'], pred, beta=2)
        tresh_max = tresh

In [25]:
tresh_max

0.05

In [6]:
pred_proba = automl.predict(df_test).data[:, 0]#[:,0]
pred = (pred_proba > tresh_max)*1
print('gini: ', round(2*roc_auc_score(df_test['target'], pred_proba)-1,3))
print('f2_score: ', round(fbeta_score(df_test['target'], pred, beta=2),3))
print('f1_score: ', round(f1_score(df_test['target'], pred),3))
print('accuracy_score: ', round(accuracy_score(df_test['target'], pred),3))
print('recall_score: ', round(recall_score(df_test['target'], pred),3))
print('precision_score: ', round(precision_score(df_test['target'], pred),3))

gini:  0.746
f2_score:  0.5
f1_score:  0.312
accuracy_score:  0.762
recall_score:  0.835
precision_score:  0.192


In [13]:
#сохраняем лучшую модель
with open('train_automl_f2_1.dat', 'wb') as f:
    dill.dump(automl,f)